# custom_aisearch_agent.py - ELI5 Walkthrough
This notebook reconstructs a proof-of-concept workflow that pairs Azure AI Search retrieval with an Agent Framework pipeline.
It mirrors the sample structure used throughout these ELI5 notebooks so you can explore each stage of the orchestration step by step.


## Big Picture
Use Azure AI Search to pull grounded context, hand the enriched payload to an Azure OpenAI agent, and surface the
final answer as the workflow output.


## Key Ingredients
- `AzureSearchRetriever` wraps the Azure AI Search SDK and keeps the index hydrated with sample data.
- `RetrieveAzureSearchContext` calls the retriever from inside a workflow executor node.
- `AnswerWithSearchContext` runs an Azure OpenAI backed agent that transforms search results into a response.
- `WorkflowBuilder` connects the retrieval and answer stages without hand-rolled orchestration code.


## Workflow Diagram
```mermaid
flowchart LR
    Q(["User Query"]) --> R[["Azure AI Search Executor"]]
    R --> A[["Agent synthesizer"]]
    A --> O(["Grounded Answer"])
```


### Step 1: Imports, configuration, and scenario overview
Load environment variables first, then pull in the Azure AI Search SDK and Agent Framework helpers used throughout the sample.


In [1]:
# Copyright (c) Microsoft. All rights reserved.
from dotenv import load_dotenv
load_dotenv()

import asyncio
import os
from typing import Any, Dict, Iterable, List

from agent_framework import (
    ChatAgent,
    Executor,
    WorkflowBuilder,
    WorkflowContext,
    handler,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import ResourceExistsError
from azure.identity import AzureCliCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchFieldDataType,
    SearchableField,
    SimpleField,
)

"""
Scenario summary:
- Use Azure AI Search to fetch relevant snippets for a user question.
- Feed those snippets into an Azure OpenAI powered agent that drafts the final answer.
- Demonstrate how execution tasks and agent tasks cooperate inside WorkflowBuilder.

Environment prerequisites (document these values in `.env.example`):
- `AZURE_SEARCH_ENDPOINT`
- `AZURE_SEARCH_API_KEY`
- `AZURE_SEARCH_INDEX_NAME`
- `AZURE_OPENAI_ENDPOINT`, `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`, `AZURE_OPENAI_API_VERSION` (consumed by AzureOpenAIChatClient)

Optional: set `SEED_SEARCH_INDEX=true` to load a small in-memory data set for quick demos.
"""


'\nScenario summary:\n- Use Azure AI Search to fetch relevant snippets for a user question.\n- Feed those snippets into an Azure OpenAI powered agent that drafts the final answer.\n- Demonstrate how execution tasks and agent tasks cooperate inside WorkflowBuilder.\n\nEnvironment prerequisites (document these values in `.env.example`):\n- `AZURE_SEARCH_ENDPOINT`\n- `AZURE_SEARCH_API_KEY`\n- `AZURE_SEARCH_INDEX_NAME`\n- `AZURE_OPENAI_ENDPOINT`, `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`, `AZURE_OPENAI_API_VERSION` (consumed by AzureOpenAIChatClient)\n\nOptional: set `SEED_SEARCH_INDEX=true` to load a small in-memory data set for quick demos.\n'

### Step 2: Wrap Azure AI Search in a lightweight helper
The helper below owns the Search clients, ensures the index schema exists, optionally seeds sample documents, and exposes a
`get_retrieval_context` method that returns a newline-joined string of document excerpts.


In [4]:
class AzureSearchRetriever:
    """Utility that manages index setup and fetches snippets using Azure AI Search."""

    def __init__(self, endpoint: str, api_key: str, index_name: str) -> None:
        credential = AzureKeyCredential(api_key)
        self.index_name = index_name
        self.index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
        self.search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

    def ensure_index(self) -> None:
        """Create the search index if it does not already exist."""
        fields = [
            SimpleField(name="id", type=SearchFieldDataType.String, key=True),
            SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),
        ]
        index = SearchIndex(name=self.index_name, fields=fields)
        try:
            self.index_client.create_index(index)
        except ResourceExistsError:
            # Index already exists; reuse it.
            return

    def seed_documents(self, documents: Iterable[Dict[str, Any]]) -> None:
        """Upload sample documents to make the demo self-contained.

        Skip if the iterable is empty to avoid unnecessary service calls.
        """
        docs = list(documents)
        if not docs:
            return
        self.search_client.upload_documents(documents=docs)

    def get_retrieval_context(self, query: str) -> str:
        """Run a simple full-text search and concatenate the snippets."""
        results = self.search_client.search(query)
        context_strings: List[str] = []
        for result in results:
            context_strings.append(f"Document: {result['content']}")
        return "".join(context_strings) if context_strings else "No results found"


SAMPLE_DOCUMENTS: List[Dict[str, Any]] = [
    {"id": "1", "content": "Contoso Banking waives monthly fees for savings accounts with automated deposits."},
    {"id": "2", "content": "Customers can reach Contoso support 24/7 via chat, phone, or the mobile app."},
    {"id": "3", "content": "The rewards credit card includes travel insurance and zero foreign transaction fees."},
]


### Step 3: Define workflow executors and bind the agent
One executor asks Azure AI Search for context, the next executor hands that context to an agent and produces the final answer.


In [6]:
class RetrieveAzureSearchContext(Executor):
    """Fetch search results for the inbound query and forward context downstream."""

    def __init__(self, retriever: AzureSearchRetriever) -> None:
        super().__init__(id="azure_ai_search_lookup")
        self.retriever = retriever

    @handler
    async def handle(self, query: str, ctx: WorkflowContext[dict[str, str], str]) -> None:
        context = self.retriever.get_retrieval_context(query)
        payload = {"query": query, "context": context}
        await ctx.send_message(payload)


class AnswerWithSearchContext(Executor):
    """Run an agent with the retrieved snippets and yield the final answer."""

    def __init__(self, agent: ChatAgent) -> None:
        super().__init__(id="answer_with_context")
        self.agent = agent

    @handler
    async def handle(self, payload: dict[str, str], ctx: WorkflowContext[str, str]) -> None:
        query = payload["query"]
        context = payload["context"]
        prompt = ("You are a helpful banking assistant. Use only the provided context to answer the question."
            "If the context says 'No results found', acknowledge that the information is unavailable."
            "Context:{context} Question: {query} Answer:"
        )
        response = await self.agent.run(prompt.format(context=context, query=query))
        if response.messages:
            answer = response.messages[-1].text
        else:
            answer = "The agent did not return a message."
        await ctx.yield_output(answer)


def build_search_workflow(retriever: AzureSearchRetriever, agent: ChatAgent):
    """Construct the two-node workflow and return it."""
    retrieve = RetrieveAzureSearchContext(retriever)
    answer = AnswerWithSearchContext(agent)
    return (
        WorkflowBuilder()
        .set_start_executor(retrieve)
        .add_edge(retrieve, answer)
        .build()
    )


### Step 4: Bootstrap the clients, optionally seed the index, and run the workflow
This cell wires everything together. The helper handles Jupyter vs. script execution so you can run it in either environment.


In [ ]:
async def main() -> None:
    """Instantiate clients, build the workflow, and execute a sample query."""
    search_endpoint = os.environ.get("AZURE_SEARCH_ENDPOINT")
    search_api_key = os.environ.get("AZURE_SEARCH_API_KEY")
    index_name = os.environ.get("AZURE_SEARCH_INDEX_NAME")

    if not all([search_endpoint, search_api_key, index_name]):
        raise RuntimeError("Missing Azure AI Search configuration. Check .env settings.")

    retriever = AzureSearchRetriever(
        endpoint=search_endpoint,
        api_key=search_api_key,
        index_name=index_name,
    )
    retriever.ensure_index()

    if os.environ.get("SEED_SEARCH_INDEX", "false").lower() in {"1", "true", "yes"}:
        retriever.seed_documents(SAMPLE_DOCUMENTS)

    chat_client = AzureOpenAIChatClient(credential=AzureCliCredential())
    agent = chat_client.create_agent(
        instructions=("You answer customer banking questions and only cite information present in the context provided."
            "Respond in 2-3 sentences."
        )
    )

    workflow = build_search_workflow(retriever, agent)
    events = await workflow.run("What perks does the rewards credit card include?")
    outputs = events.get_outputs()
    if outputs:
        print(outputs[-1])
    else:
        print("Workflow completed without outputs.")


loop = asyncio.get_event_loop()
if loop.is_running():
    await main()
else:
    asyncio.run(main())
